In [20]:
from deap import base, creator, tools
import random
import itertools
import string

# define the signal and meaning spaces
signals = list(string.ascii_lowercase)  # All possible signals
#meanings = [0, 1, 2, 3]         # All possible meanings
meanings = list(range(0,4))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax, local_state=None)

toolbox = base.Toolbox()

toolbox.register("attr_signal", random.choice, signals)
toolbox.register("attr_meaning", random.choice, meanings)
toolbox.register("attr_local_state", random.randint, 0, 3)
toolbox.register("attr_global_state", random.randint, 0, 3)

GLOBAL = toolbox.attr_global_state()
print("Global State: ", GLOBAL)

def create_individual():
    trans_genes = [toolbox.attr_signal() for _ in meanings]  
    recept_genes = [toolbox.attr_meaning() for _ in signals]  
    individual = creator.Individual(trans_genes + recept_genes)
    individual.local_state = toolbox.attr_local_state() 
    return individual

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


GLOBAL_STATE = random.randint(0, 3)

def pairwise_communication(speaker, listener):

    # speaker encodes signals
    local_meaning = speaker.local_state
    global_meaning = GLOBAL

    semantic_local = speaker[local_meaning]
    semantic_global = speaker[global_meaning]

    # listener decodes local signal
    local_index = 0
    for signal in signals:
        if semantic_local == signal:
            decoded_local = listener[local_index + len(meanings)]
        local_index += 1


    # if semantic_local == 'a':
    #     decoded_local = listener[4]
    # elif semantic_local == 'b':
    #     decoded_local = listener[5]
    # elif semantic_local == 'c':
    #     decoded_local = listener[6]
    # else:
    #     decoded_local = listener[7]

    # listener decodes global signal
    global_index = 0
    for signal in signals:
        if semantic_global == signal:
            decoded_global = listener[global_index + len(meanings)]
        global_index += 1




    # if semantic_global == 'a':
    #     decoded_global = listener[4]
    # elif semantic_global == 'b':
    #     decoded_global = listener[5]
    # elif semantic_global == 'c':
    #     decoded_global = listener[6]
    # else:
    #     decoded_global = listener[7]

    fitness = 0
    if local_meaning == decoded_local:
        fitness += 1
    if global_meaning == decoded_global:
        fitness += 1
    return fitness   

# evaluate group
def evaluate_group(group):
    for ind in group:
        ind.fitness.values = (0,)  # reset fitness
    
    for speaker, listener in itertools.permutations(group, 2):
        fitness_bonus = pairwise_communication(speaker, listener)
        speaker.fitness.values = (speaker.fitness.values[0] + fitness_bonus,)
        listener.fitness.values = (listener.fitness.values[0] + fitness_bonus,)

# create pop
population = toolbox.population(n=100)

GROUP_SIZE = 5

# eval groups
for i in range(0, len(population), GROUP_SIZE):
    group = population[i:i + GROUP_SIZE]
    evaluate_group(group)

for ind in population:
    print(ind, ind.fitness.values)


Global State:  0
['t', 'l', 'o', 't', 3, 2, 3, 1, 1, 0, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 1, 3, 3, 2, 0, 3, 1, 0, 3] (2.0,)
['s', 't', 'o', 'z', 0, 1, 1, 0, 3, 2, 2, 2, 3, 1, 0, 2, 2, 1, 3, 3, 0, 3, 2, 2, 0, 1, 1, 1, 2, 3] (4.0,)
['g', 'g', 'w', 'e', 1, 1, 3, 0, 0, 3, 0, 1, 3, 0, 3, 3, 0, 0, 0, 0, 3, 0, 2, 3, 2, 0, 1, 0, 1, 0] (3.0,)
['d', 'n', 'i', 'o', 3, 0, 3, 2, 2, 0, 1, 0, 2, 2, 1, 2, 0, 3, 2, 3, 3, 3, 2, 3, 0, 0, 0, 0, 2, 1] (5.0,)
['v', 'w', 'q', 'd', 3, 0, 1, 0, 1, 2, 2, 1, 0, 3, 3, 1, 3, 1, 0, 0, 1, 2, 0, 2, 3, 2, 2, 2, 0, 3] (6.0,)
['q', 'p', 'n', 'k', 0, 1, 1, 3, 2, 2, 1, 3, 3, 1, 1, 0, 1, 1, 0, 3, 1, 1, 0, 2, 0, 2, 3, 0, 0, 3] (1.0,)
['k', 'd', 'n', 'e', 2, 2, 1, 3, 0, 0, 3, 2, 2, 2, 3, 1, 2, 3, 3, 3, 3, 3, 1, 0, 2, 2, 2, 0, 2, 0] (5.0,)
['f', 'k', 'b', 'r', 2, 3, 1, 0, 1, 0, 0, 2, 2, 0, 0, 2, 0, 1, 0, 3, 2, 3, 3, 0, 1, 0, 1, 3, 3, 2] (6.0,)
['v', 'k', 'y', 'y', 1, 3, 2, 3, 0, 0, 3, 2, 1, 0, 1, 3, 2, 1, 2, 3, 3, 0, 1, 2, 0, 2, 1, 1, 0, 0] (4.0,)
['e', 'x', 'f', 'c', 2, 3, 3,